In [18]:
# 夏目漱石の小説『吾輩は猫である』の文章（neko.txt）をMeCabを使って形態素解析し，
# その結果をneko.txt.mecabというファイルに保存せよ．
# このファイルを用いて，以下の問に対応するプログラムを実装せよ．

import requests

r = requests.get("https://nlp100.github.io/data/neko.txt", auth=('user','pass'))
r.status_code
r.headers['content-type']
r.encoding
with open("../data/neko.txt",mode='w') as f:
    f.write(r.text)